# **2.2 The Idea of Elimination**

### **Elimination**

- **Goal:** Produce an **upper triangular system**, which makes finding the unknowns easier.  
- Solve unknowns using **back substitution**.  
- To eliminate $x$: Subtract a multiple of equation 1 from equation 2.  

**Definitions:**  
- **Pivot:** First nonzero entry in the row used for elimination.  
- **Multiplier:** Entry to eliminate divided by pivot: $l_{ij} = e / p$.

In [23]:
def matrix_elimination_multiplier(A):
    pivot = A[0][0]
    to_elim = A[1][0]
    multiplier = to_elim/pivot
    return multiplier

A = [[1, -2],[3, 2]]
matrix_elimination_multiplier(A)

3.0

### **Breakdown of Elimination**

Elimination normally produces pivots leading to a solution, but **failure is possible**. Three scenarios where elimination could fail:

1. **Permanent failure (no solution):**  
   - **Row picture:** Parallel lines with no meeting point.  
   - **Column picture:** Two or more left-hand columns point in the same direction, but the right-hand column points differently → no solution.  

2. **Infinitely many solutions:**  
   - **Row picture:** Parallel lines coincide → every point satisfies both equations.  
   - **Column picture:** $n$ equations but fewer than $n$ pivots.

3. **Temporary failure (zero in pivot):**  
   - **Row picture:** Normal; solution exists.  
   - **Column picture:** Pivot is zero (not allowed). Fix by **row exchange** or **permutation**.

In [36]:
def elimination_solution_type(A, b, tol=1e-10):
    A = A.astype(float).copy()
    b = b.astype(float).copy()
    n, m = A.shape

    # Forward Elimination
    row = 0
    for col in range(m):
        # Find pivot
        pivot_row = None
        for r in range(row, n):
            if abs(A[r, col]) > tol:
                pivot_row = r
                break
        if pivot_row is None:
            continue # Columns is all zeros, move to next
        # Swap if needed
        if pivot_row != row:
            A[[row, pivot_row]] = A[[pivot_row, row]]
            b[[row, pivot_row]] = b[[pivot_row, row]]

        # Eliminate below
        for r in range(row+1, n):
            factor = A[r, col]/A[row,col] # multiplier
            A[r, col:] -= factor * A[row, col:] # subract the multiplier times the pivot to the current row
            b[r] -= factor * b[row] # subract the multiplier times the pivot to the current value in b

        row += 1

    # Check for inconsistency (row of zeros in A but nonzero in b)
    for r in range(n):
        if np.all(np.abs(A[r]) < tol) and abs(b[r]) > tol:
            return "No solution"

    # Check if rank < number of variables
    rank = np.sum([np.any(np.abs(A[r]) > tol) for r in range(n)])
    if rank < m:
        return "Infinite solution"

    return "Unique solution"
            
                

A1 = np.array([[2, 1], [4, 2]])
b1 = np.array([3, 6])
print(elimination_solution_type(A1, b1))

A2 = np.array([[1, 1], [1, 1]])
b2 = np.array([2, 3])
print(elimination_solution_type(A2, b2))

A3 = np.array([[1, 2], [3, 4]])
b3 = np.array([5, 11])
print(elimination_solution_type(A3, b3))

Infinite solution
No solution
Unique solution



### **Three Equations in Three Unknowns**

Let:

$$
\begin{cases}
2x + 4y - 2z = 2 \\
4x + 9y - 3z = 8 \\
-2x - 3y + 7z = 1
\end{cases}
$$

**Step 1:** Eliminate row2 col1 and row3 col1 using first pivot $row_1 col_1$:

$$
\begin{cases}
2x + 4y - 2z = 2 \\
\ \ \ y + z = 4 \\
\ -y + 5z = 12
\end{cases}
$$

**Step 2:** Use new pivot $row_2 col_2$ to eliminate $row_3 col_2$:

$$
\begin{cases}
2x + 4y - 2z = 2 \\
\ \ \ y + z = 4 \\
\ \ \ 4z = 8
\end{cases}
$$

**Step 3:** Back substitution:

$$
z = 2, \quad y + z = 4 \Rightarrow y = 2, \quad 2x + 4y - 2z = 2 \Rightarrow x = -1
$$

**Step 4:** Verify solution using matrix form:

$$
A x = 
\begin{bmatrix} 2 & 4 & -2 \\ 4 & 9 & -3 \\ -2 & -3 & 7 \end{bmatrix} 
\begin{bmatrix}-1 \\ 2 \\ 2 \end{bmatrix} 
= 
\begin{bmatrix} 2 \\ 8 \\ 10 \end{bmatrix} = b
$$

- The numbers $x, y, z$ multiply columns 1, 2, 3 in $Ax = b$ and in the triangular $Ux = c$.


### **Elimination from $A$ to $U$**

- For $n \times n$ systems, elimination proceeds **column by column**:

1. **Column 1:** Use first equation to create zeros below the first pivot.  
2. **Column 2:** Use new equation 2 to create zeros below the second pivot.  
3. **Columns 3 to $n$:** Continue to find all $n$ pivots and form upper triangular $U$.

In [26]:
import numpy as np

def elimination_breakdown(A, b):
    A = A.astype(float).copy()
    b = b.astype(float).copy()
    n = len(A)

    print("Initial System")
    for i in range(n):
        print(f"{A[i]} | {b[i]}")
    print("-"*30)

    for i in range(n):
        pivot = A[i][i]
        if pivot == 0:
            raise ValueError("Pivot is zero, need row swapping!")
        print(f"Eliminating column {i} using pivot of row {i}")

        for j in range(i+1, n):
            factor = A[j, i] / pivot # multiplier
            A[j, i:] -= factor * A[i, i:] # subract the multiplier times the pivot to the current row
            print(f"\nStep: eliminate row {j} using row {i} * {factor:.2f}")
            for k in range(n):
                print(f"{A[k]} | {b[k]}")
        print("-"*30)

    print("Forward elimination complete. Upper-triangular A and modified by b:")
    for i in range(n):
        print(f"{A[i]} | {b[i]}")

    return

A = np.array([[2, 4, -2],
             [4, 9, -3],
             [-2, -3, 7]], dtype=float)
b = np.array([2, 8, 10], dtype=float)
elimination_breakdown(A, b)

Initial System
[ 2.  4. -2.] | 2.0
[ 4.  9. -3.] | 8.0
[-2. -3.  7.] | 10.0
------------------------------
Eliminating column 0 using pivot of row 0

Step: eliminate row 1 using row 0 * 2.00
[ 2.  4. -2.] | 2.0
[0. 1. 1.] | 8.0
[-2. -3.  7.] | 10.0

Step: eliminate row 2 using row 0 * -1.00
[ 2.  4. -2.] | 2.0
[0. 1. 1.] | 8.0
[0. 1. 5.] | 10.0
------------------------------
Eliminating column 1 using pivot of row 1

Step: eliminate row 2 using row 1 * 1.00
[ 2.  4. -2.] | 2.0
[0. 1. 1.] | 8.0
[0. 0. 4.] | 10.0
------------------------------
Eliminating column 2 using pivot of row 2
------------------------------
Forward elimination complete. Upper-triangular A and modified by b:
[ 2.  4. -2.] | 2.0
[0. 1. 1.] | 8.0
[0. 0. 4.] | 10.0


In [43]:
def back_substitution(A, b, tol=1e-10):
    A = A.astype(float).copy()
    b = b.astype(float).copy()
    n, m = A.shape

    # Forward Elimination
    row = 0
    for col in range(m):
        # Find pivot
        pivot_row = None
        for r in range(row, n):
            if abs(A[r, col]) > tol:
                pivot_row = r
                break
        if pivot_row is None:
            continue # Columns is all zeros, move to next
        # Swap if needed
        if pivot_row != row:
            A[[row, pivot_row]] = A[[pivot_row, row]]
            b[[row, pivot_row]] = b[[pivot_row, row]]

        # Eliminate below
        for r in range(row+1, n):
            factor = A[r, col]/A[row,col] # multiplier
            A[r, col:] -= factor * A[row, col:] # subract the multiplier times the pivot to the current row
            b[r] -= factor * b[row] # subract the multiplier times the pivot to the current value in b

        row += 1

    # Check for inconsistency (row of zeros in A but nonzero in b)
    for r in range(n):
        if np.all(np.abs(A[r]) < tol) and abs(b[r]) > tol:
            return "No solution", None

    # Check if rank < number of variables
    rank = np.sum([np.any(np.abs(A[r]) > tol) for r in range(n)])
    if rank < m:
        return "Infinite solution", None

    # Back-substitution (unique solution)
    x = np.zeros(m)
    print("Back-substitution steps:")
    for i in reversed(range(n)):
        nz_cols = np.where(np.abs(A[i]) > tol)[0]
        if len(nz_cols) == 0:
            continue # This should not happen here
        col = nz_cols[0]
        sum_known = np.dot(A[i,col+1:], x[col+1:])
        x[col] = (b[i] - sum_known) / A[i,col]
        print(f"x[{col}] = (b[{i}] - sum_known) / A[{i}, {col}] = ({b[i]} - {sum_known}) / {A[i, col]} = {x[col]}")
    return "Unique solution", x
            
                

A1 = np.array([[2, 1], [4, 2]])
b1 = np.array([3, 6])
stype, x = back_substitution(A1, b1)
print(stype, x)

A2 = np.array([[1, 1], [1, 1]])
b2 = np.array([2, 3])
stype, x = back_substitution(A2, b2)
print(stype, x)

A3 = np.array([[1, 2], [3, 4]])
b3 = np.array([5, 11])
stype, x = back_substitution(A3, b3)
print(stype, x)

Infinite solution None
No solution None
Back-substitution steps:
x[1] = (b[1] - sum_known) / A[1, 1] = (-4.0 - 0.0) / -2.0 = 2.0
x[0] = (b[0] - sum_known) / A[0, 0] = (5.0 - 4.0) / 1.0 = 1.0
Unique solution [1. 2.]


**Key Ideas**

1. Linear system $Ax = b$ becomes **upper triangular** $Ux = c$ after elimination.  
2. Subtract $l_{ij}$ times equation $j$ from equation $i$ to make the $(i, j)$ entry zero.  
3. Multiplier: $l_{ij} = (\text{entry to eliminate in row } i) / (\text{pivot in row } j)$. Pivots **cannot be zero**.  
4. If pivot is zero, **exchange rows** if a nonzero entry exists below.  
5. Solve $Ux = c$ using **back substitution** (start at the bottom).  
6. Permanent breakdown → $Ax = b$ has **no solution** or **infinitely many solutions**.